In [48]:
import warnings
import sys
sys.path.append('../')
from Notebooks.helper import load_env

warnings.filterwarnings('ignore')
load_env()

Import all the required dependencies

In [49]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
import os
import pandas as pd
import json

In [50]:
prompt = """
Answer the question below:
Question: {{ query }}
"""
prompt_builder = PromptBuilder(template=prompt)
generator = OpenAIGenerator(model="gpt-4")

query_pipeline = Pipeline()
query_pipeline.add_component("prompt", prompt_builder)
query_pipeline.add_component("generator", generator)

query_pipeline.connect("prompt", "generator")

🚅 Components
  - prompt: PromptBuilder
  - generator: OpenAIGenerator
🛤️ Connections
  - prompt.prompt -> generator.prompt (str)

In [51]:
df = pd.read_csv('gics-map-2018.csv')
for index, row in df.iterrows():
    # Access the values of each column in the current row
    sub_industry_description = row['SubIndustryDescription']
    question = f"Give me an unformatted list of Climate related risks and mitigations for a company who's industry is classified as {sub_industry_description}? Provide response in a single JSON array of objects, each object has an attribute risk and an attribute mitigation"
    result = query_pipeline.run(
        {
            "prompt": {"query": question},
        }
    )
    print('Risks - '+result['generator']['replies'][0])
    df.at[row.name,'Climate Risk Mitigation'] = result['generator']['replies'][0]

    # question = f"Give me an unformatted list of Risk mitigations for the following Risks: {result['generator']['replies'][0]}? Provide response in this exact JSON format: [\"mitigation\",\"mitigation\"]"
    # result = query_pipeline.run(
    #     {
    #         "prompt": {"query": question},
    #     }
    # )
    print('Mitigations - '+result['generator']['replies'][0])

    
    df.to_csv('Gics_modified_gpt4.csv')

Risks - [
  {
    "risk": "Higher frequency of extreme weather events",
    "mitigation": "Incorporate environmental hazard assessment into project planning and management."
  },
  {
    "risk": "Rise in temperature affecting operational efficiency",
    "mitigation": "Improve infrastructure and technology for temperature resilience."
  },
  {
    "risk": "Regulatory impacts related to emissions and environmental impacts",
    "mitigation": "Invest in cleaner, low-carbon technologies and enhance environmental management systems."
  },
  {
    "risk": "Supply chain disruption due to weather events",
    "mitigation": "Diversify supply chain and improve logistics."
  },
  {
    "risk": "Water scarcity affecting drilling operations",
    "mitigation": "Invest in water efficient technologies and reuse/recycling strategies."
  },
  {
    "risk": "Damage to infrastructure due to extreme weather",
    "mitigation": "Regular assessment and maintenance of infrastructure."
  },
  {
    "risk": "

Set up the document ingestion pipeline

In [52]:
print(result["generator"]["replies"][0])

[
  {
    "risk": "Increased insurance cost due to extreme weather events",
    "mitigation": "Developing and adopting robust disaster management strategies"
  },
  {
    "risk": "Damage to properties from flooding due to rising sea levels",
    "mitigation": "Investing in adaptive and resilient infrastructure"
  },
  {
    "risk": "Lower property values due to climate change concerns",
    "mitigation": "Focusing on sustainable and eco-friendly property development"
  },
  {
    "risk": "Decreased demand for properties in areas affected by climate change",
    "mitigation": "Diversifying property portfolio across different geographical locations"
  },
  {
    "risk": "Increased maintenance costs due to extreme weather conditions",
    "mitigation": "Considering climate change factors in maintenance planning"
  },
  {
    "risk": "Risk of litigation or regulatory fines due to lack of compliance with environmental laws",
    "mitigation": "Staying updated and compliance with all relevan

In [53]:
industry_analysis_pipeline = Pipeline()